In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time
from sqlalchemy import false
from sympy import true



from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service

class Voltaire:
    driver = ''
    ndc=''
    mdp=''
    csv = "fichier.csv"
    df = pd.read_csv(csv, sep=";")
    df = pd.DataFrame(df)
    phrase = ''
    entrainement = []

    def __init__(self,ndc,mdp):
        self.ndc = ndc
        self.mdp = mdp

    def goTo (self):
        #options.add_argument("--headless")
        options = Options()
        options.add_argument("start-maximized")
        options.add_argument("disable-infobars")
        options.add_argument("--disable-extensions");
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()),chrome_options=options)
        self.driver.get("https://www.projet-voltaire.fr/")

    def connect(self):
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.ID, 'cmpbntyestxt'))).click()
        self.driver.find_element_by_xpath('//*[@id="authenticateOption"]').click()
        self.driver.implicitly_wait(10) # seconds
        #NDC
        self.driver.find_element_by_xpath('//*[@id="login-username"]').send_keys(self.ndc)
        self.driver.implicitly_wait(10) # seconds
        #MDP
        self.driver.find_element_by_xpath('//*[@id="login-pwd"]').send_keys(self.mdp)
        self.driver.implicitly_wait(10) # seconds
        WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'recaptcha-checkbox-border'))).click()
        
    def makeTraining(self):
        self.entrainement = []
        entrainementtemp = WebDriverWait(volt.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'app-circle-title')))
        counter=0
        for val in entrainementtemp:
            t = val.find_element_by_xpath("..")
            parent = t.get_attribute("class")
            if ((parent == "app-circle green active")):
                print(val.text)
                self.entrainement.append(val)
            counter += 1

        
    def goTraining(self,indice):

        self.entrainement[indice].click()
        # for train in self.entrainement:
        #     train.click()


    def findUsedTo(self):
        time.sleep(1)
        self.phrase = ""

        indice = 0
        for val in WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan'))):
            if val.text=="":
                self.phrase += " "
            else:
                self.phrase += val.text

            indice += 1

    def mkSentenceCourriel(self):
        time.sleep(1)
        self.phrase = ""

        indice = 0
        for val in WebDriverWait(volt.driver, 20).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'pointAndClickSpan'))):
            if val.text=="":
                self.phrase += " "
            else:
                self.phrase += val.text

            indice += 1

    def mkSentencePronominalCsv(self):
        time.sleep(1)
        self.phrase = ""

        test = WebDriverWait(volt.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'instructions')))
        for i in test:
            self.phrase+= i.text

        # print(self.phrase)
        indice = 0
        for val in WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan'))):

            #print(val.text)
            if val.text=="":
                self.phrase += " "
            else:
                self.phrase += val.text
            #print(val.text)
            #print(indice)
            indice += 1
        print(self.phrase)


    def findNormalized(self):
        time.sleep(1)
        self.phrase = ""
        for val in self.driver.find_elements_by_class_name('pointAndClickSpan'):
            #print(val.text)
            self.phrase += val.text
            self.phrase += ' '
        print(self.phrase)

    def checkCsv(self):
        try:
            if self.df[self.df["phrase"].eq(self.phrase)]["value"].item():
                #print(1)
                #alors on clique pas de faute
                WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_question_suivante"]'))).click()
                result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            else:
                #print(2)
                #print(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())
                val = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
                try:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())].click()
                    #print("gclické")
                except:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())+1].click()
                    #print("print n'a pas reussi a cliquer")
        except:
            WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.CLASS_NAME, 'noMistakeButton'))).click()

            #result = driver.find_element(by=By.CLASS_NAME, value='answerStatusBarLabel1')
            result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            if (result=="Mauvaise réponse"):
                #print(3)

                phrase = self.driver.find_elements(by=By.CLASS_NAME, value='pointAndClickSpan')
                counter=0
                bool = True
                for val in phrase:
                    t = val.find_element_by_xpath("..")
                    parent = t.get_attribute("class")
                    if ((parent == "answerWord") & (bool)):
                        toinput = counter
                        df2 = {"phrase":self.phrase, "value":False ,"word":toinput}
                        self.df = self.df.append(df2, ignore_index=True)
                        self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")
                        #print("Parent class attribute: " + t.get_attribute("class")) 
                        bool = False
                    counter += 1
            else:
                print(4)
                df2 = {"phrase":self.phrase, "value":True ,"word":""}
                self.df = self.df.append(df2, ignore_index=True)
                self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")

        
        WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div[2]/div[3]/div/div[3]/div[2]/button'))).click()
        
    def checkCsvPronom(self):
        try:
            if self.df[self.df["phrase"].eq(self.phrase)]["value"].item():
                print(1)
                #alors on clique pas de faute
                WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_question_suivante"]'))).click()
                result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            else:
                print(2)
                print(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())
                val = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
                try:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())].click()
                    print("gclické")
                except:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())+1].click()
                    print("print n'a pas reussi a cliquer")
        except:
            val = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
            val[0].click()
            
            result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            if (result=="Mauvaise réponse"):
                print(3)

                phrase = self.driver.find_elements(by=By.CLASS_NAME, value='pointAndClickSpan')
                counter=0
                bool = True
                for val in phrase:
                    t = val.find_element_by_xpath("..")
                    parent = t.get_attribute("class")
                    if ((parent == "answerWord") & (bool)):
                        toinput = counter
                        df2 = {"phrase":self.phrase, "value":False ,"word":toinput}
                        self.df = self.df.append(df2, ignore_index=True)
                        self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")
                        #print("Parent class attribute: " + t.get_attribute("class")) 
                        bool = False
                    counter += 1
            else:
                print(4)
                df2 = {"phrase":self.phrase, "value":True ,"word":""}
                self.df = self.df.append(df2, ignore_index=True)
                self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")

        
        WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[5]/div[2]/div[3]/div/div[3]/div[2]/button'))).click()

    def checkCsvCourriel(self):
        try:
            if self.df[self.df["phrase"].eq(self.phrase)]["value"].item():
                #alors on clique pas de faute
                WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_question_suivante"]'))).click()
                result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            else:
                try:
                    val = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
                except:
                    val = WebDriverWait(self.driver, 20).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
                try:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())].click()
                    #print("gclické")
                except:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())+1].click()
                    #print("print n'a pas reussi a cliquer")
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'clickonmail-next-button'))).click()
        except:
            WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_question_suivante"]'))).click()

            #result = driver.find_element(by=By.CLASS_NAME, value='answerStatusBarLabel1')
            result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            if (result=="Mauvaise réponse"):
                print(3)
                phrase = self.driver.find_elements(by=By.CLASS_NAME, value='pointAndClickSpan')
                counter=0
                bool = True
                for val in phrase:
                    t = val.find_element_by_xpath("..")
                    parent = t.get_attribute("class")
                    if ((parent == "answerWord") & (bool)):
                        toinput = counter
                        df2 = {"phrase":self.phrase, "value":False ,"word":toinput}
                        self.df = self.df.append(df2, ignore_index=True)
                        self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")
                        #print("Parent class attribute: " + t.get_attribute("class")) 
                        bool = False
                    counter += 1
            else:
                df2 = {"phrase":self.phrase, "value":True ,"word":""}
                self.df = self.df.append(df2, ignore_index=True)
                self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")

            
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'clickonmail-next-button'))).click()
            #si jamais y'a un "la prochaine question portera sur"
        try:
            WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'understoodButton'))).click()
        except:
            print("nothing")

    def mkSentenceQuatre(self):
        self.phrase = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'qcc-instructions')))[0].text
        self.phrase += WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'qcc-question')))[0].text

    def mkSentenceQuatreNiveau5(self):
        self.phrase = WebDriverWait(self.driver, 20).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'qcc-mail-instructions')))[0].text
        for elem in WebDriverWait(self.driver, 20).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'pointAndClickSpan'))):
            self.phrase += elem.text


    def quatre(self,numer):
        try:   
            val = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'qc-proposal-button')))
            for values in val:
                if values.text == str(self.df[self.df["phrase"].eq(self.phrase)]["word"].item()):
                    values.click()
        except:
            val = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'qc-proposal-button')))
            val[0].click()

            result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            if (result=="Mauvaise réponse"):
                print(3)
                phrase = self.driver.find_elements(by=By.CLASS_NAME, value='qc-proposal-button')
                bool = True
                for val in phrase:
                    t = val.find_element_by_xpath("..")
                    parent = t.get_attribute("class")
                    if ((parent == "qc-proposal correct locked") & (bool)):
                        df2 = {"phrase":self.phrase, "value":False ,"word":val.text}
                        self.df = self.df.append(df2, ignore_index=True)
                        self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")
                        #print("Parent class attribute: " + t.get_attribute("class")) 
                        bool = False
            else:
                print(4)
                df2 = {"phrase":self.phrase, "value":True ,"word":val[0].text}
                self.df = self.df.append(df2, ignore_index=True)
                self.df.to_csv(self.csv,encoding='utf-8', index=False, sep=";")
        if numer==1:
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'qcc-next'))).click()
        else:
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'qcc-mail-next'))).click()
            
            

    def testBlanc(self):
        try:
            if self.df[self.df["phrase"].eq(self.phrase)]["value"].item():
                #print(1)
                #alors on clique pas de faute
                WebDriverWait(self.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_question_suivante"]'))).click()
                result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
            else:
                #print(2)
                #print(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())
                val = WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
                try:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())].click()
                    #print("gclické")
                except:
                    val[int(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())+1].click()
                    #print("print n'a pas reussi a cliquer")
        except:
            print("I did not found")


    def checkCouilles(self):
            try:
                #print(df[df["phrase"].eq(phrase)]["value"].item())
                print(2)
                print(self.df[self.df["phrase"].eq(self.phrase)]["word"].item())
                for val in WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan'))):
                    try:
                        if (val.text == self.df[self.df["phrase"].eq(self.phrase)]["word"].item()):
                            val.click()
                            print("gclické")
                    except:
                        print("print n'a pas reussi a cliquer")
            except:
                WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))[0].click()

                #result = driver.find_element(by=By.CLASS_NAME, value='answerStatusBarLabel1')
                result = WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
                if (result=="Mauvaise réponse"):
                    print(3)
                    elemCorrect = self.driver.find_element(by=By.CLASS_NAME, value='answerWord')
                    ListeElemCorrect = elemCorrect.text

                    # if "'" in toinput:
                    #     print("présence '")
                    #     toinput = toinput.split("'")
                    #     if toinput[0] == "":
                    #         toinput = toinput[1]
                    #     else:
                    #         toinput = toinput[0]

                    # if "‑" in toinput:
                    #     print("présence ‑")
                    #     toinput = toinput.split("‑")
                    #     if toinput[0] == "":
                    #         toinput = toinput[1]
                    #     else:
                    #         toinput = toinput[0]
                    # if "," in toinput:
                    #     print("présence ,")
                    #     toinput = toinput.split(",")
                    #     if toinput[0] == "":
                    #         toinput = toinput[1]
                    #     else:
                    #         toinput = toinput[0]
                    # if "." in toinput:
                    #     print("présence '")
                    #     toinput = toinput.split(".")
                    #     if toinput[0] == "":
                    #         toinput = toinput[1]
                    #     else:
                    #         toinput = toinput[0]
                    
                    df2 = {"phrase":self.phrase, "value":False ,"word":ListeElemCorrect}
                    self.df = self.df.append(df2, ignore_index=True)
                    self.df.to_csv("niveau1-8-10.csv",encoding='utf-8', index=False, sep=";")
                else:
                    print(4)
                    df2 = {"phrase":self.phrase, "value":True ,"word":ListeElemCorrect}
                    self.df = self.df.append(df2, ignore_index=True)
                    self.df.to_csv("niveau1-8-10.csv",encoding='utf-8', index=False, sep=";")

            
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'nextButton'))).click()
            
            try:
                WebDriverWait(self.driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME, 'understoodButton'))).click()
            except:
                print("nothing")


    def understood(self):
        if (len(self.driver.find_elements_by_class_name('understoodButton')) > 0):
            self.driver.find_element_by_class_name('understoodButton').click()
            #WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[7]/div/div/div[3]/button'))):
            items = self.driver.find_elements_by_class_name('buttonOk')
            for item in items:
                item.click()

            try:
                #on doit check si c'est correct ou si c'est faux
                self.driver.find_element_by_xpath('/html/body/div[7]/div/div/div/div[5]/button[3]').click()
            except:
                print("il n'y a pas de button jabandone")
        else:
            pass

    def understoodPronom(self):
        if (len(self.driver.find_elements_by_class_name('understoodButton')) > 0):
            self.driver.find_element_by_class_name('understoodButton').click()
            
    def doExercise(self, method, name):
        if method == "id":
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.ID, name))).click()
        elif method == "class":
            WebDriverWait(self.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, name))).click()
        elif method == "anyid":
            WebDriverWait(volt.driver, 20).until(EC.visibility_of_any_elements_located((By.ID, name)))[0].click()
        elif method == "anyclass":
            WebDriverWait(volt.driver, 20).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, name)))[0].click()
        else:
            print("rien")



    def realize(self,type):
        bExercise = True
        if type == 1:
            while bExercise:
                try:
                    volt.findUsedTo()
                    volt.checkCsv()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        elif type==2:
            while bExercise:
                try:
                    volt.findUsedTo()
                    volt.testBlanc()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        elif type==3:
            #Verbes pronominaux I
            while bExercise:
                try:
                    volt.mkSentencePronominalCsv()
                    volt.checkCsvPronom()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        elif type==4:
            #Verbes pronominaux II
            while bExercise:
                try:
                    try:
                        volt.mkSentencePronominalCsv()
                        volt.checkCsvPronom()
                        
                    except:
                        volt.findUsedTo()
                        volt.checkCsv()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        elif type==5:
            #courriel niv 1,3,4 (mail classiques)
            while bExercise:
                try:
                    volt.mkSentenceCourriel()
                    volt.checkCsvCourriel()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        elif type==6:
            #courriel niv 2 (quatre questions)
            while bExercise:
                try:
                    volt.mkSentenceQuatre()
                    volt.quatre(1)
                    volt.understoodPronom()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        elif type==7:
            while bExercise:
                try:
                    try:
                        volt.mkSentenceQuatre()
                        volt.quatre(1)
                        volt.understoodPronom()    
                    except:
                        volt.mkSentenceCourriel()
                        volt.checkCsvCourriel()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        elif type==8:
            #courriel niveau 5
            while bExercise:
                try:
                    volt.mkSentenceQuatreNiveau5()
                    volt.quatre(2)
                    volt.understoodPronom()
                except:
                    WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_apprentissage_autres_niveaux"))).click()
                    bExercise = False
        else:
            print('jsp')


        
#go to proj voltaire
volt = Voltaire("YOUR NAME ACCOUNT","YOUR PASSWORD")
volt.goTo()


In [ ]:
#Connect
volt.connect()


#volt.orth()
#volt.courriel()

# WebDriverWait(volt.driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.XPATH,"//iframe[starts-with(@name, 'a-')] and starts-with(@src, 'https://www.google.com/recaptcha')]"))); 
# elem = WebDriverWait(volt.driver, 20).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"div.recaptcha-checkbox-checkmark"))); 

#WebDriverWait(self.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))


In [ ]:
volt.makeTraining()
# volt.goTraining(0)

#res = WebDriverWait(volt.driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//button[contains( text( ), 'Lancer')]")))

exercise = "btn_home_module_lancer_"


# # #lvl 1-10
# for numero in range(1,11):
#     try:
#         volt.doExercise("id",exercise+str(numero) )
#         volt.realize(1)
#         print("Niveau "+str(numero)+" termine")
#     except:
#         print("Niveau "+str(numero)+" deja realise")
# #vali finale
# try:
#     volt.doExercise("id","validation-activity-cell-launch-button")
#     volt.realize(1)
#     print("validation finale termine")
# except:
#     print("validation finale deja realise")
# #testblanc
# try:
#     volt.doExercise("class","blank-test-activity-launch-button")
#     volt.realize(2)
#     print("test blanc termine")
# except:
#     print("test blanc deja realise")


# WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, 'btn_sortir'))).click()


# #niveau orthotypographie
# WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Orthotypographie')]"))).click()

# #lvl 1-4
# for numero in range(1,5):
#     try:
#         volt.doExercise("anyid",exercise+str(numero) )
#         volt.realize(1)
#         print("Niveau "+str(numero)+" termine")
#     except:
#         print("Niveau "+str(numero)+" deja realise")
# #testblanc

# #je dois tester mes connaissances button ok
# try:
#     volt.doExercise("anyclass","blank-test-activity-launch-button")
#     volt.realize(2)
#     print("test blanc termine")
# except:
#     print("test blanc deja realise")
# WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, 'btn_sortir'))).click()


#niveau excellence
#WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//*[contains(text(), 'Excellence')]"))).click()

#lvl 1-pronom
# for numero in range(1,15):
    
#     try:
#         if numero==13:
#             volt.doExercise("anyid",exercise+str(numero) )
#             volt.realize(3)
#         elif numero==14:
#             volt.doExercise("anyid",exercise+str(numero) )
#             volt.realize(4)
#         else:
#             volt.doExercise("anyid",exercise+str(numero) )
#             volt.realize(1)
#             print("Niveau "+str(numero)+" termine")
#     except:
#         print("Niveau "+str(numero)+" deja realise")


#retour menu
#WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.ID, "btn_home_sortir"))).click()
#WebDriverWait(volt.driver, 20).until(EC.visibility_of_any_elements_located((By.ID, "btn_home_module_lancer_1")))[0].click()

volt.goTraining(1)

#lvl 1-5
for numero in range(1,6):

    
    

    if (numero == 5):
        for vk in range(2):
            try:
                volt.doExercise("class","validation-activity-cell-launch-button")
                try:
                    volt.realize(7)
                except:
                    print("")   
            except:
                print("pas de validation 3 et 4")

    volt.doExercise("id",exercise+str(numero))
    if (numero == 1 | numero == 3| numero == 4):
        try:
            volt.realize(5)
            print("Niveau "+str(numero)+" termine")
        except:
            print("Niveau "+str(numero)+" deja realise")
    elif(numero == 2):
        try:

            volt.realize(6)
            print("Niveau "+str(numero)+" termine")
        except:
            print("Niveau "+str(numero)+" deja realise")

    if (numero == 5):
        try:
            volt.doExercise("class","validation-activity-cell-launch-button")
            try:
                volt.realize(8)
            except:
                print("")   
        except:
            print("pas de validation 5")



#validation-activity-cell-launch-button


#if volt.driver.find_elements_by_css_selector('#btn_apprentissage_autres_niveaux') condition

#btn_apprentissage_autres_niveaux

# try:
#     
#btn_apprentissage_autres_niveaux
# except:
#     print("non")   

# for i in range(500):
#      volt.findUsedTo()
#      volt.checkCsv()



In [ ]:


#volt.doExercise("id",exercise+str(1) )



In [ ]:
volt.realize(7)

In [ ]:

for i in range(200):
    volt.mkSentenceCourriel()
    volt.checkCsvCourriel()

# volt.quatre()



#print(volt.df[volt.df["phrase"].eq(volt.phrase)]["word"].item())

In [ ]:
#make sentence
for i in range(300):
    volt.mkSentenceQuatre()
    volt.quatre()
    volt.understoodPronom()


# val = WebDriverWait(volt.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'qc-proposal-button')))
# len(val)
# print(val[0].text)
# print(val[1].text)
# print(val[2].text)
# print(val[3].text)


In [ ]:
volt.mkSentenceCourriel()


if volt.df[volt.df["phrase"].eq(volt.phrase)]["value"].item():
    #alors on clique pas de faute
    WebDriverWait(volt.driver, 20).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="btn_question_suivante"]'))).click()
    result = WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'answerStatusBarLabel1'))).text
else:

    val = WebDriverWait(volt.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
    try:
        val[int(volt.df[volt.df["phrase"].eq(volt.phrase)]["word"].item())].click()
        #print("gclické")
    except:
        val[int(volt.df[volt.df["phrase"].eq(volt.phrase)]["word"].item())+1].click()
        #print("print n'a pas reussi a cliquer")
WebDriverWait(volt.driver, 20).until(EC.visibility_of_element_located((By.CLASS_NAME, 'clickonmail-next-button'))).click()




In [ ]:
#print(volt.df[volt.df["phrase"].eq(volt.phrase)]["value"].item())
#val = WebDriverWait(volt.driver, 20).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
# val = WebDriverWait(volt.driver, 20).until(EC.visibility_of_any_elements_located((By.CLASS_NAME, 'pointAndClickSpan')))
# for va in val:
#     print(va.text)

volt.realize(8)